In [20]:
from realkd.rules import RuleBoostingEstimator, XGBRuleEstimator, logistic_loss
from realkd.search import search_methods
from numba import njit, parallel_chunksize
import pandas as pd
import sortednp as snp
import doctest

from collections import defaultdict, deque
from sortedcontainers import SortedSet
from math import inf
import numpy as np
from heapq import heappop, heappush
from numpy import array
from bitarray import bitarray
import collections.abc

from math import inf
from numpy import arange, argsort, array, cumsum, exp, full_like, log2, stack, zeros, zeros_like
from pandas import qcut, Series
import time
from sklearn.base import BaseEstimator, clone

from realkd.search import Conjunction, Context, KeyValueProposition, Constraint
from bitarray.util import subset

from realkd.logic import Conjunction, Constraint, KeyValueProposition, TabulatedProposition

from numba import njit, prange
from numba.typed import List

import pandas as pd

RNG = np.random.default_rng(seed=0)

@njit
def intersect_sorted_arrays(A, B):
  """
  Returns the sorted intersection of A and B
  - Assumes A and B are sorted
  - Assumes A and B each have no duplicates
  """
  i = 0
  j = 0
  intersection = List()

  while i < len(A) and j < len(B):
      if A[i] == B[j]:
          intersection.append(A[i])
          i += 1
          j += 1
      elif A[i] < B[j]:
          i += 1
      else:
          j += 1
  return np.asarray(intersection)

def rand_array(size, alpha=0.2):
    n, k = size
    d = np.arange(n*k)
    RNG.shuffle(d)
    d = (d < alpha*len(d)).astype(int)
    return d.reshape(n, k)

In [10]:
# n = length of data
# m = width (columns) of data
ns = np.arange(50, 10051, 1000)
ms = np.arange(50, 501, 50)

alpha = 0.5

d = {}
for m in ms:
    for n in ns:
        X = rand_array((n, m), alpha=alpha)
        true_weights = RNG.random(m) * 10
        y = X @ true_weights + RNG.random(n)
        y = np.sign(y - y.mean())
        dfX = pd.DataFrame(data=X, index=None, columns=[f'x{i}' for i in range(X.shape[1])])
        dfy = pd.Series(data=y)
        d[(m, n)] = (dfX, dfy)

In [24]:
def get_objective_function(X, y):
    loss = logistic_loss # TODO: Generalize
    reg = 1.0 # TODO: Generalize
    predictions = zeros_like(y) # TODO: Generalize
    g = array(loss.g(y, predictions))
    h = array(loss.h(y, predictions))
    r = g / h
    order = argsort(r)[::-1]
    g = g[order]
    h = h[order]
    n = y.shape[0]

    @njit
    def objective_function(ext):
        if len(ext) == 0:
            return -inf
        g_q = g[ext]
        h_q = h[ext]
        return g_q.sum() ** 2 / (2 * n * (reg + h_q.sum()))

    return objective_function

@njit
def find_best_new_index(current_intent, current_extent, extents, initial_value, n, objective_function):
    best_i, best_ext = None, current_extent
    value = initial_value
    for i in range(n):
        for index in current_intent:
            if index == i:
                continue
        new_extent = intersect_sorted_arrays(current_extent, extents[i])
        new_value = objective_function(new_extent)
        if new_value > value:
            value = new_value
            best_ext = new_extent
            best_i = i
    return (best_i, best_ext, value)

@njit(parallel=True)
def find_best_new_index_parallel(current_intent, current_extent, extents, initial_value, n, objective_function):
    best_i, best_ext = None, current_extent
    value = initial_value
    for i in range(n):
        for index in current_intent:
            if index == i:
                continue
        new_extent = intersect_sorted_arrays(current_extent, extents[i])
        new_value = objective_function(new_extent)
        if new_value > value:
            value = new_value
            best_ext = new_extent
            best_i = i
    return (best_i, best_ext, value)


In [26]:
data = d[(ms[0], ns[0])]
ctx = Context.from_df(data[0])
objective_function = get_objective_function(data[0], data[1])
initial_extent = np.array(ctx.extension([]))
n = ctx.n
extents = List(ctx.extents)
attributes = ctx.attributes

initial_value = objective_function(initial_extent)

find_best_new_index(initial_extent, initial_extent, extents, initial_value, n, objective_function)
find_best_new_index_parallel(initial_extent, initial_extent, extents, initial_value, n, objective_function)

find_best_new_index_parallel.parallel_diagnostics(level=4)
# %timeit -o find_best_new_index(initial_extent, initial_extent, extents, initial_value, n, objective_function)
# %timeit -o find_best_new_index_parallel(initial_extent, initial_extent, extents, initial_value, n, objective_function)

/home/alastair/personal/rawork/realkd.py/ra-venv/lib/python3.8/site-packages/numba/core/typed_passes.py:329: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../tmp/ipykernel_4406/429975193.py", line 39:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


 
 Parallel Accelerator Optimizing:  Function find_best_new_index_parallel, 
/tmp/ipykernel_4406/429975193.py (39)  
No source available
--------------------------------- Fusing loops ---------------------------------
Attempting fusion of parallel loops (combines loops with similar properties)...
----------------------------- Before Optimisation ------------------------------
--------------------------------------------------------------------------------
------------------------------ After Optimisation ------------------------------
Parallel structure is already optimal.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
 
---------------------------Loop invariant code motion---------------------------
Allocation hoisting:
No allocation hoisting found

Instruction hoisting:
No instruction hoisting found
------------------------------------------------------------------------